In [ ]:
from transformers import pipeline
from nltk import sent_tokenize
import nltk
import torch
from glob import glob

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/burii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
## Load the model

model_name = "facebook/bart-large-mnli"
device = 0 if torch.cuda.is_available() else 'cpu'

In [9]:
def load_model(device):
    theme_classifier = pipeline(
        "zero-shot-classification", 
        model=model_name, 
        device=device)
    return theme_classifier

In [10]:
theme_classifier = load_model(device)

/home/burii/Documents/projects/nlp/naruto/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
theme_list = ["friendship","hope","sacrifice","battle","self development","betrayal","love","dialogue"]

In [12]:
theme_classifier(
    "I gave him a right hook then a left jab",
    theme_list,
    multi_label=True
)

{'sequence': 'I gave him a right hook then a left jab',
 'labels': ['battle',
  'self development',
  'hope',
  'sacrifice',
  'dialogue',
  'betrayal',
  'love',
  'friendship'],
 'scores': [0.9121253490447998,
  0.47500181198120117,
  0.08781816065311432,
  0.04500045254826546,
  0.020132921636104584,
  0.012040478177368641,
  0.0042923795990645885,
  0.002817221451550722]}

In [ ]:
#Load Dataset